In [1]:
#Opening Datafile in csv format
import pandas as pd
data=pd.read_csv("Walmart.csv")
df=data.copy()

In [2]:
#Checking Data Types
datatypes=data.dtypes
datatypes

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [3]:
#Reset Data Type
df['Date']=pd.to_datetime(df['Date'])
datatypes=df.dtypes
datatypes

Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

In [4]:
#Checking Missing Values
def Completness_test(df):
    missing_values_sum=pd.isnull(df).any(axis=1).sum()
    total_sum=df.any(axis=1).sum()
    Completeness_score=(missing_values_sum/total_sum)*100
    Complete_score=(float(Completeness_score))
    if Complete_score!=0:
        print(Complete_score)
        print(" \nCount total NaN at each column in a DataFrame : \n\n",df.isnull().sum()) 
    else:
        print("The data is complete. There is no missing values")
CT=Completness_test(df)

The data is complete. There is no missing values


In [5]:
#Checking Duplicates
def Duplication_test(df):
    duplicate_values_sum=df.duplicated().any(axis=0).sum()
    total_sum=df.any(axis=1).sum()
    Duplication_percent=(duplicate_values_sum/total_sum)*100
    Duplication_percent=float(Duplication_percent)
    if Duplication_percent!=0:
        print(Duplication_percent)
        print(" \nCount total NaN at each column in a DataFrame : \n\n",df.duplicated().sum()) 
    else:
        print("The data is unique. There is no duplicate values")
Duplication_test(df)

The data is unique. There is no duplicate values


In [6]:
# Checking outliers
import numpy as np
def Outlier_Detect(data):
    data_mean, data_std = np.mean(data), np.std(data)
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = [x for x in data if x < lower or x > upper]
    outliers_removed = [x for x in data if x >= lower and x <= upper]
    Percentage=(len(outliers)/(len(outliers)+len(outliers_removed)))*100
    print('Total outliers percent: %d' % Percentage)
for i in df:
    if df[i].dtype=='int64':
        print(i)
        Outlier_Detect(df[i])

Store
Total outliers percent: 0
Holiday_Flag
Total outliers percent: 6


In [7]:
df['Holiday_Flag'].unique()

array([0, 1], dtype=int64)

In [8]:
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['year'] = df['Date'].dt.year
df['quarter'] = df['Date'].dt.quarter
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,day,month,year,quarter
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,2,5,2010,2
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106,2,12,2010,4
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,19,2,2010,1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,26,2,2010,1
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106,3,5,2010,2


In [9]:
#Importing SQL in Python
#! pip install psycopg2-binary
#! pip install snowflake-connector-python
#! pip install mysql-connector-python

In [10]:
#Converting DataFrame to SQL
import pandas as pd
import sqlite3
conn = sqlite3.connect('test_database_2') 
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS sales (Store, Date, Weekly_Sales, Holiday_Flag, Temperature, Fuel_Price, CPI, Unemployment,day, month,year,quarter)')
conn.commit()
df.to_sql('sales', conn, if_exists='replace', index = False)

In [11]:
#Highest sales according to year and shops which made it
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,quarter,Store,max(Weekly_Sales)
          FROM sales
          GROUP BY year,quarter
          ORDER BY (year) ASC
          ''')
df2= pd.DataFrame(c.fetchall(), columns = ['Year','Quarter','Store','Weekly_Sales'])
print (df2)

    Year  Quarter  Store  Weekly_Sales
0   2010        1     14    2495630.51
1   2010        2     14    2623469.95
2   2010        3     14    2370116.52
3   2010        4     14    3818686.45
4   2011        1      4    2316495.56
5   2011        2     20    2351143.07
6   2011        3     20    2546123.78
7   2011        4      4    3676388.98
8   2012        1      4    2427640.17
9   2012        2     20    2565259.92
10  2012        3      4    2283540.30
11  2012        4     20    2462978.28


In [12]:
#Highest sales
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          Store, max(Weekly_Sales)
          FROM sales
          ''')
print(c.fetchall())

[(14, 3818686.45)]


In [13]:
#Highest sales
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          quarter,year,sum(Weekly_Sales)
          FROM sales
          WHERE year=2012
          GROUP BY quarter,year
          ''')
df3= pd.DataFrame(c.fetchall(), columns = ['Quarter','Year','Weekly_Sales'])
print (df3)

   Quarter  Year  Weekly_Sales
0        1  2012  5.457900e+08
1        2  2012  6.172141e+08
2        3  2012  5.559950e+08
3        4  2012  2.811338e+08


In [14]:
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,sum(Weekly_Sales),Store
          FROM sales
          WHERE year=2012
          GROUP BY Store
          ''')
df4= pd.DataFrame(c.fetchall(), columns = ['Year','Weekly_Sales','Store'])
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,sum(Weekly_Sales)
          FROM sales
          WHERE year=2012
          ''')
d1=(c.fetchall())
print(d1)
#Top 10% contributor
df4['Percent']=(df4['Weekly_Sales']/2000132859.3500023)*100
df4.sort_values("Percent", axis = 0, ascending = False,inplace = True, na_position ='last')
df5=(df4.head(10%len(df4['Percent'])))
print(df5)

[(2012, 2000132859.3500023)]
    Year  Weekly_Sales  Store   Percent
3   2012   92771189.24      4  4.638251
19  2012   89827709.38     20  4.491087
12  2012   86707455.02     13  4.335085
1   2012   81496695.37      2  4.074564
9   2012   78228616.94     10  3.911171
13  2012   77441398.26     14  3.871813
26  2012   71920056.12     27  3.595764
0   2012   68202058.02      1  3.409876
5   2012   66315047.00      6  3.315532
38  2012   65885662.85     39  3.294064


In [15]:
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,sum(Weekly_Sales),Store
          FROM sales
          WHERE year=2011
          GROUP BY Store
          ''')
df6= pd.DataFrame(c.fetchall(), columns = ['Year','Weekly_Sales','Store'])
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,sum(Weekly_Sales)
          FROM sales
          WHERE year=2011
          ''')
d7=(c.fetchall())
print(d7)
#Top 10% contributor
df6['Percent']=(df6['Weekly_Sales']/2448200007.3499975)*100
df6.sort_values("Percent", axis = 0, ascending = False,inplace = True, na_position ='last')
df7=(df6.head(10%len(df4['Percent'])))
print(df7)

[(2011, 2448200007.3499975)]
    Year  Weekly_Sales  Store   Percent
3   2011  1.110923e+08      4  4.537713
19  2011  1.098370e+08     20  4.486439
13  2011  1.060963e+08     14  4.333644
12  2011  1.045375e+08     13  4.269974
9   2011  9.891689e+07     10  4.040393
1   2011  9.860788e+07      2  4.027771
26  2011  9.192268e+07     27  3.754705
0   2011  8.092192e+07      1  3.305364
5   2011  8.052876e+07      6  3.289305
38  2011  7.577760e+07     39  3.095237


In [16]:
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,sum(Weekly_Sales),Store
          FROM sales
          WHERE year=2010
          GROUP BY Store
          ''')
df8= pd.DataFrame(c.fetchall(), columns = ['Year','Weekly_Sales','Store'])
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          year,sum(Weekly_Sales)
          FROM sales
          WHERE year=2010
          ''')
d8=(c.fetchall())
print(d8)
#Top 10% contributor
df8['Percent']=(df6['Weekly_Sales']/2288886120.41)*100
df8.sort_values("Percent", axis = 0, ascending = False,inplace = True, na_position ='last')
df9=(df8.head(10%len(df4['Percent'])))
print(df9)

[(2010, 2288886120.41)]
    Year  Weekly_Sales  Store   Percent
3   2010  9.568047e+07      4  4.853553
19  2010  1.017331e+08     20  4.798710
13  2010  1.054622e+08     14  4.635280
12  2010  9.527274e+07     13  4.567178
9   2010  9.447220e+07     10  4.321617
1   2010  9.527786e+07      2  4.308117
26  2010  9.001318e+07     27  4.016044
0   2010  7.327883e+07      1  3.535428
5   2010  7.691232e+07      6  3.518251
38  2010  6.578228e+07     39  3.310676


In [17]:
conn = sqlite3.connect('test_database_2')
c.execute('''
          SELECT
          quarter,year,sum(Weekly_Sales),Store
          FROM sales
          WHERE year=2012
          GROUP BY Store,quarter
          ''')
df10= pd.DataFrame(c.fetchall(), columns = ['Quarter','Year','Weekly_Sales','Store'])
print (df10)

     Quarter  Year  Weekly_Sales  Store
0          1  2012   18951097.69      1
1          2  2012   21036965.58      1
2          3  2012   18633209.98      1
3          4  2012    9580784.77      1
4          1  2012   22543946.63      2
..       ...   ...           ...    ...
175        4  2012    2022983.83     44
176        1  2012    9073434.67     45
177        2  2012   10278900.05     45
178        3  2012    8851242.32     45
179        4  2012    4520053.13     45

[180 rows x 4 columns]


In [18]:
L1=list(df5['Store'])
L2=list(df7['Store'])
L3=list(df9['Store'])
print(L1,L2,L3)

[4, 20, 13, 2, 10, 14, 27, 1, 6, 39] [4, 20, 14, 13, 10, 2, 27, 1, 6, 39] [4, 20, 14, 13, 10, 2, 27, 1, 6, 39]


In [19]:
df.head(3)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,day,month,year,quarter
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,2,5,2010,2
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106,2,12,2010,4
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,19,2,2010,1


In [20]:
def Pearson_correlation(X,Y):
    if len(X)==len(Y):
        Sum_xy = sum((X-X.mean())*(Y-Y.mean()))
        Sum_x_squared = sum((X-X.mean())**2)
        Sum_y_squared = sum((Y-Y.mean())**2)       
        corr = Sum_xy / np.sqrt(Sum_x_squared * Sum_y_squared)
    return corr
for i in (df['Temperature'],df['Fuel_Price'],df['CPI'],df['Unemployment']):  
    print(Pearson_correlation(df['Weekly_Sales'],i))

-0.06381001317946962
0.009463786314475168
-0.07263416204017649
-0.10617608965795382
